In [2]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread, pprint
import pandas as pd
import numpy as np

In [3]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('bplinebot-3ccea59ad6d6.json', scope)
client = gspread.authorize(creds)
pp = pprint.PrettyPrinter()

In [66]:
#sheet = client.open('AbbokIncomeAssesmentV03').get_worksheet(2)
spread = client.open('userCheckin')

In [67]:
spread.worksheets()

[<Worksheet 'sum' id:576946557>,
 <Worksheet 'log' id:0>,
 <Worksheet 'type' id:252782580>,
 <Worksheet 'userStatus' id:1060831238>,
 <Worksheet 'key' id:1108603979>]

In [68]:
sheet = spread.worksheet('log')

In [69]:
list_of_lists = sheet.get_all_values()

In [72]:
df = pd.DataFrame(list_of_lists[1:], columns = list_of_lists[0])
#dfc['Time_of_Sail'] = pd.to_datetime(dfc['Time_of_Sail'],format= '%H:%M:%S' ).dt.time
#df['TIME'] = pd.to_datetime(df['TIME']).dt.time
df['TIME'] = pd.to_datetime(df['TIME']).values.astype(np.int64)
df['DATE'] = pd.to_datetime(df['DATE']).dt.date
df['USER_ID'] = df['USER_ID'].astype(int)
df['TYPE'] = df['TYPE'].astype(int)
df = df.set_index('ID')
df.head(10)

,DATE,TIME,USER_ID,TYPE
ID,,,,
1,2019-06-06,1563936300000000000,1,1
2,2019-06-06,1563936360000000000,1,0
3,2019-06-11,1563983460000000000,3,1
4,2019-06-11,1563983520000000000,3,0
5,2019-06-11,1563983700000000000,2,1
6,2019-06-11,1563984120000000000,1,1
7,2019-06-11,1563984240000000000,3,1
8,2019-06-11,1563984240000000000,2,0
9,2019-06-11,1563984240000000000,4,1


In [55]:
df[(df['USER_ID'] == 1) & (df['TYPE'] == 1)]

,DATE,TIME,USER_ID,TYPE
ID,,,,
1,2019-06-06,02:45:00,1,1
6,2019-06-11,16:02:00,1,1
20,2019-06-14,14:38:00,1,1
28,2019-06-17,13:33:00,1,1
33,2019-06-18,13:21:00,1,1
44,2019-06-19,13:56:00,1,1
56,2019-06-21,12:53:00,1,1
64,2019-06-24,14:10:00,1,1
69,2019-06-24,18:47:00,1,1


In [77]:
df.groupby(['USER_ID', 'TYPE'])['TIME'].mean()

USER_ID  TYPE
1        0       1563990204000000000
         1       1563975939000000000
2        0       1563987292499999744
         1       1563963699999999744
3        0       1563988763333332992
         1       1563966290000000000
4        0       1563995480869565184
         1       1563970899130434816
6        0       1563987229411764480
         1       1563961750588235264
7        0       1563996982500000000
         1       1563961477500000000
Name: TIME, dtype: int64

"มีเบอร์ ['BEST', 'TAAN', 'SNOOK', 'TEAM', 'BAM', 'BOW', 'TOEY']"